  # TheFuzz

<div align="center">


  <a href="https://github.com/1treu1/Deduplicacion-de-Datos/tree/main/TheFuzz" target="_blank">
    <img width="1024", src="https://m.media-amazon.com/images/S/pv-target-images/39981144ca9d92fcfa857223ea889663d35999d5fae146d42f658cf7c49f025a.jpg" width="800" height="400"></a>


<br>
  <a href="https://colab.research.google.com/drive/1LMZnbyKUBBGIzB7fPmd2dhI6KNpJ_MHO?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://github.com/1treu1/Deduplicacion-de-Datos/tree/main/TheFuzz"><img src="https://img.shields.io/badge/github-Open In Github-brightgreen.svg" alt="Open In Github"></a>
</br>
Esta libreria permite encontrar registros duplicados usando la distancia de Levenstein.
</div>


* Levenstein Distance: https://en.wikipedia.org/wiki/Levenshtein_distance

## Paso 1: Instalar Librerias

In [1]:
!pip install thefuzz
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00


# Paso 2: Crear Base de Datos de Ejemplo

Crearemos a 1000 clientes de una empresa, sus atributos son Nombre, Apellido, DNI y Fecha de Nacimiento. Para esto, usaremos la libreria Faker, que permite generar nombres y datos aleatorios de lo que queramos :3

In [2]:
import pandas as pd
from faker import Faker

# Crear una instancia de Faker
fake = Faker("es_ES")
Faker.seed(222)
# Crear una lista vacía para almacenar los usuarios
usuarios = []

# Generar 1000 usuarios aleatorios
for _ in range(1000):
    nombre = fake.first_name()
    apellido = fake.last_name()
    dni = fake.unique.random_number(digits=8)
    fecha_nacimiento = fake.date_of_birth(minimum_age=18, maximum_age=90)
    usuarios.append((nombre, apellido, dni, fecha_nacimiento))

# Crear un dataframe a partir de la lista de usuarios
df = pd.DataFrame(usuarios, columns=['Nombre', 'Apellido', 'DNI', 'Fecha de nacimiento'])

# Mostrar los primeros registros del dataframe
print(df.head())

    Nombre   Apellido       DNI Fecha de nacimiento
0  Melania     Campos  31574101          1971-06-18
1  Mariano     Angulo   3918234          1998-05-16
2  Jacinto    Salgado  99348528          1992-11-17
3   Víctor  Benavides   2180849          1946-09-10
4   Silvio      Belda  65820934          2002-12-17


# Paso 3: Ahora crearemos algunos duplicados:

In [3]:
df1 = df.sample(frac=0.2, random_state=333) #Creando 200 duplicados
df1.index = range(len(df1))
df1.shape

(200, 4)

In [4]:
print("Datos originales", df.shape)
print("Datos duplicados", df1.shape)

Datos originales (1000, 4)
Datos duplicados (200, 4)


# Paso 4: Limpieza
* Ahora vamos a hacer una limpieza de los datos

In [7]:
dfA = df.copy()
dfB = df1.copy()

dfA.shape, dfB.shape

((1000, 4), (200, 4))

* Convertimos cada columna en tipo str

In [8]:
dfA['Nombre'] = dfA['Nombre'].astype(str)
dfA['Apellido'] = dfA['Apellido'].astype(str)
dfA['Fecha de nacimiento'] = dfA['Fecha de nacimiento'].astype(str)
dfB['Nombre'] = dfB['Nombre'].astype(str)
dfB['Apellido'] = dfB['Apellido'].astype(str)
dfB['Fecha de nacimiento'] = dfB['Fecha de nacimiento'].astype(str)

* Pasamos todas las cadenas a minusculas

In [9]:
dfA['Nombre'] = dfA['Nombre'].str.lower()
dfA['Apellido'] = dfA['Apellido'].str.lower()
dfB['Nombre'] = dfB['Nombre'].str.lower()
dfB['Apellido'] = dfB['Apellido'].str.lower()

* Eliminamos carecteres especiales

In [23]:
CharEspetial = ["á","é","í","ó","ú","à","è","ì","ò","ù","-","/","  "]
Char = ["a","e","i","o","u","a","e","i","o","u","",""," "]
for i in zip(CharEspetial, Char):
  dfA['Nombre'] = dfA['Nombre'].str.replace(i[0], i[1])
  dfA['Apellido'] = dfA['Apellido'].str.replace(i[0], i[1])
  dfB['Nombre'] = dfB['Nombre'].str.replace(i[0], i[1])
  dfB['Apellido'] = dfB['Apellido'].str.replace(i[0], i[1])

RB = pd.concat([dfA,dfB])
RB.index = range(len(RB))
RB.shape

(1200, 4)

# Paso 5: Hallando los duplicados
* Usaremos la distancia de Levenstein para encontrar la similitud entre las cadenas

In [51]:
from thefuzz import fuzz
import time
st = time.time()
def remove_duplicates(df, threshold=90):
    duplicates = set()
    processed = []
    dup = []
    ndup = []
    scores = []
    scores2 = []
    raw = []
    for i, row in df.iterrows():
        if i not in duplicates:
            processed.append(row)

            for j, other_row in df.iterrows():
                if i != j and j not in duplicates:
                    score = fuzz.ratio(row["Nombre"], other_row["Nombre"])
                    score2 = fuzz.ratio(row["Apellido"], other_row["Apellido"])

                    if score >= threshold and score2 >=threshold:
                        duplicates.add(j)
                        dup.append(other_row)
                        dup.append(row)
                        #ndup.append(row)
                        #scores.append(score)
                        #scores2.append(score2)

    et = time.time()
    print("Duracion", et-st)
    df1 = pd.DataFrame(dup)
    #df1['Scores Nombre'] = scores
    #df1['Scores Apellido'] = scores
    df2 = pd.DataFrame(ndup)
    #df2['Scores'] = scores
    df3 = pd.DataFrame(processed)
    #df3['Scores'] = scores


    return df1, df2, df3 # pd.DataFrame(processed)


dfOut, dfOut2, raw = remove_duplicates(RB, 90)

Duracion 74.43184757232666


In [53]:
dfOut #Son los duplicados que encontro en el dataframe RB

,Nombre,Apellido,DNI,Fecha de nacimiento
1055,herminio,pedraza,93914566,1966-10-31
5,herminio,pedraza,93914566,1966-10-31
1083,santiago,amador,26218645,2002-04-04
8,santiago,amador,26218645,2002-04-04
1062,roman,velasco,11893770,1973-02-21
...,...,...,...,...
990,xiomara,sosa,58853655,1941-05-08
1152,leonor,cazorla,85863928,1984-05-27
993,leonor,cazorla,85863928,1984-05-27
1197,maria angeles,caballero,36058442,1942-03-22


* Podemos ver que encontró 202 duplicados, un total de 204 registros. De hecho fue capaz de encontrar 2 registros duplicados más, que no creamos al principio, sino que fue generado en la libreria.

* Gracias por ver 🥰